In [1]:
import pandas as pd
import numpy as np

In [2]:
# Custom functions
from core.data_manipulations import merge_main_and_group, aggregate_biosphere_facility_groups

In [3]:
metallican_path = r'C:\Users\mp_ma\OneDrive - polymtl\POST_DOC\CODE\metallican_db'

# Import MetalliCan tables

In [4]:
main_table = pd.read_csv(metallican_path + r'\database\CSV\main_table.csv')
production_table = pd.read_csv(metallican_path + r'\database\CSV\production_table.csv')
tech_attributes_table = pd.read_csv(metallican_path + r'\database\CSV\tech_attributes_table.csv')
env_table = pd.read_csv(metallican_path + r'\database\CSV\environmental_flows_table.csv')
technosphere_table = pd.read_csv(metallican_path + r'\database\CSV\materials_energy_table.csv')
archetypes_table = pd.read_csv(metallican_path + r'\database\CSV\archetypes_table.csv')
land_table = pd.read_csv(metallican_path + r'\database\CSV\land_occupation_table.csv')
intensity_table = pd.read_csv(metallican_path + r'\database\CSV\intensity_table.csv')
waste_table = pd.read_table(metallican_path + r'\database\CSV\waste_table.csv', sep=',')

In [5]:
substances_table = pd.read_csv(metallican_path + r'\database\CSV\substances_table.csv')

In [6]:
intensity_table_nrj = intensity_table[intensity_table['type'] == 'Energy']
intensity_table_nrj = intensity_table_nrj[~intensity_table_nrj['intensity_id'].str.startswith('INT-CMP')]

In [7]:
# Let's remove ClimateTRACE data from production and env tables
production_table = production_table[~production_table['source_id'].str.startswith('Jolleys, M. et al (2024). Mineral Extraction sector: Mining and Quarrying Emissions from Copper, Iron, Bauxite, Rock and Sand, Hypervine, UK, Climate TRACE Emissions Inventory')]
env_table = env_table[~env_table['source_id'].str.startswith('Jolleys, M. et al (2024). Mineral Extraction sector: Mining and Quarrying Emissions from Copper, Iron, Bauxite, Rock and Sand, Hypervine, UK, Climate TRACE Emissions Inventory')]

In [76]:
production_table = merge_main_and_group(production_table, main_table, cols_to_add=['facility_name', 'facility_group_name'])
ta_table = merge_main_and_group(tech_attributes_table, main_table, cols_to_add=['facility_name', 'facility_group_name'])

In [77]:
production_table.to_csv(r'data\MetalliCan\pre_cleaned_data\production_table.csv', index=False, encoding='utf-8-sig')
ta_table.to_csv(r'data\MetalliCan\pre_cleaned_data\ta_table.csv', index=False, encoding='utf-8-sig')

# Harmonization of environmental flow units

In [8]:
# Let's remove ClimateTRACE data
env_table = env_table[~env_table['source_id'].str.startswith('Jolleys, M. et al (2024). Mineral Extraction sector: Mining and Quarrying Emissions from Copper, Iron, Bauxite, Rock and Sand, Hypervine, UK, Climate TRACE Emissions Inventory')]

## Harmonize units for the same substance_id

In [9]:
unit_mapping = {
'7732-18-5' : 'm3',
'NA - M10' : 'tonnes',
'NA - M09' : 'tonnes',
'NA - M16' : 'tonnes',
'630-08-0' : 'tonnes',
'NA - NOx' : 'tonnes',
'NA - 08' : 'kg',
'NA - 10' : 'tonnes',
'NA - 02' : 'kg',
'NA - M08' : 'tonnes',
'NA - GHG' : 'tCO2eq',
'NA - 12': 'kg',
'NA - SOx': 'tonnes'
}

In [10]:
conversion_factors = {
    ('kg', 'tonnes'): 1/1000,
    ('kg', 't'): 1/1000,
    ('tonnes', 'kg'): 1000,
    ('t', 'kg'): 1000,
    ('tonnes', 't'): 1,
    ('t', 'tonnes'): 1,
    ('ML', 'm3'): 1000,
    ('m3', 'ML'): 1/1000,
    ('L', 'm3'): 1/1000,
    ('m3', 'L'): 1000,
    ('Mm3', 'm3'): 1000000,
    ('km3', 'm3'): 1000,
    ('ktCO2eq', 'tCO2eq'): 1000,
    ('tCO2eq', 'ktCO2eq'): 1/1000
}


In [11]:
def update_units_and_values(row):
    substance_id = row['substance_id']
    current_unit = row['unit']
    value = row['value']

    if substance_id in unit_mapping:
        target_unit = unit_mapping[substance_id]

        if current_unit != target_unit:
            conversion_key = (current_unit, target_unit)
            if conversion_key in conversion_factors:
                value = value * conversion_factors[conversion_key]
            # Si la conversion n'est pas définie, la valeur reste inchangée

        current_unit = target_unit  # Mettre à jour l'unité

    return pd.Series({'unit': current_unit, 'value': value})


env_table['value'] = pd.to_numeric(env_table['value'], errors='coerce')
mask = env_table['substance_id'].isin(unit_mapping.keys())
env_table.loc[mask, ['unit', 'value']] = env_table.loc[mask].apply(update_units_and_values, axis=1)

In [13]:
#env_table = merge_main_and_group(env_table, main_table)

In [12]:
env_table

,env_id,year,compartment_name,substance_id,flow_direction,release_pathway,unit,value,comment,main_id,facility_group_id,company_id,source_id
28,npri-1568-2023-1,2023,Air,NA - 16,Emission,Stack Emissions,tonnes,0.63600,NaN,ON-MAIN-1f126a43,NaN,CMP-6265c407,https://www.canada.ca/en/environment-climate-c...
29,npri-1568-2023-2,2023,Air,NA - 02,Emission,Stack Emissions,kg,0.11600,NaN,ON-MAIN-1f126a43,NaN,CMP-6265c407,https://www.canada.ca/en/environment-climate-c...
30,npri-1568-2023-3,2023,Air,630-08-0,Emission,Stack Emissions,tonnes,76.20800,NaN,ON-MAIN-1f126a43,NaN,CMP-6265c407,https://www.canada.ca/en/environment-climate-c...
31,npri-1568-2023-4,2023,Air,NA - 04,Emission,Stack Emissions,tonnes,0.00105,NaN,ON-MAIN-1f126a43,NaN,CMP-6265c407,https://www.canada.ca/en/environment-climate-c...
32,npri-1568-2023-5,2023,Air,NA - 05,Emission,Stack Emissions,kg,0.18000,NaN,ON-MAIN-1f126a43,NaN,CMP-6265c407,https://www.canada.ca/en/environment-climate-c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,ENV-9de9bb0d-2023-8,2023,Air,NA - M16,Emission,NaN,tonnes,0.99000,NaN,QC-MAIN-9de9bb0d,NaN,CMP-3d2c4955,SRC_WesdomeGoldMinesLtd_2023-ESG-Data-Tables
5148,ENV-9de9bb0d-2023-9,2023,Water,7732-18-5,Withdrawal,NaN,m3,168115.00000,Water withdrawal,QC-MAIN-9de9bb0d,NaN,CMP-3d2c4955,SRC_WesdomeGoldMinesLtd_2023-ESG-Data-Tables
5149,ENV-9de9bb0d-2023-10,2023,Water,7732-18-5,Withdrawal,NaN,m3,168115.00000,Water withdrawal|Freshwater withdrawn,QC-MAIN-9de9bb0d,NaN,CMP-3d2c4955,SRC_WesdomeGoldMinesLtd_2023-ESG-Data-Tables
5150,ENV-9de9bb0d-2023-11,2023,Water,7732-18-5,Consumption,NaN,m3,82144.00000,Total water consumption,QC-MAIN-9de9bb0d,NaN,CMP-3d2c4955,SRC_WesdomeGoldMinesLtd_2023-ESG-Data-Tables


## Aggregate data at facility-group level

In [13]:
env_table = aggregate_biosphere_facility_groups(env_table, remove_individuals='True')

In [14]:
env_table.to_excel(r'data/MetalliCan/pre_cleaned_data/biosphere_df.xlsx', index=False)

# Harmonization of technosphere units

In [15]:
# Separe energy and material
energy_df = technosphere_table[technosphere_table['flow_type'] == 'Energy']
material_df = technosphere_table[technosphere_table['flow_type'] == 'Material use']

## Energy flows

In [16]:
from core.conversion_functions import standardize_energy_to_MJ

{'M_concentrate_t': 18000.0, 'M_metal_in_ore_t': 5000.0, 'M_metal_in_concentrate_t': 4500.0, 'Yield_fraction': 0.018, 'Tailings_grade': 0.0004999999999999999}


In [17]:
energy_df_sd = standardize_energy_to_MJ(energy_df)

In [18]:
energy_df_sd

,technosphere_id,year,flow_type,subflow_type,unit,value,comment,main_id,facility_group_id,company_id,source_id,value_MJ,unit_source,assumption_note,unit_standard,needs_factor
0,TECH-857b7b89-2023-1,2023,Energy,Acetylene,GJ,18.475651,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,SRC_NewmontCorporation_2023-Performance-data,1.847565e+04,direct_unit,gj→MJ factor=1000.0,MJ,False
1,TECH-857b7b89-2023-2,2023,Energy,Aviation fuel,GJ,72676.110790,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,SRC_NewmontCorporation_2023-Performance-data,7.267611e+07,direct_unit,gj→MJ factor=1000.0,MJ,False
2,TECH-857b7b89-2023-3,2023,Energy,Diesel,GJ,287042.447232,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,SRC_NewmontCorporation_2023-Performance-data,2.870424e+08,direct_unit,gj→MJ factor=1000.0,MJ,False
3,TECH-857b7b89-2023-4,2023,Energy,Gasoline,GJ,13568.450000,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,SRC_NewmontCorporation_2023-Performance-data,1.356845e+07,direct_unit,gj→MJ factor=1000.0,MJ,False
4,TECH-857b7b89-2023-5,2023,Energy,Propane,GJ,42071.041300,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,SRC_NewmontCorporation_2023-Performance-data,4.207104e+07,direct_unit,gj→MJ factor=1000.0,MJ,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,TECH-7607a50e-2023-3,2023,Energy,Diesel,GJ,140100.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,SRC_AlamosGoldInc_Data_2023,1.401000e+08,direct_unit,gj→MJ factor=1000.0,MJ,False
227,TECH-7607a50e-2023-4,2023,Energy,Gasoline,GJ,2124.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,SRC_AlamosGoldInc_Data_2023,2.124000e+06,direct_unit,gj→MJ factor=1000.0,MJ,False
228,TECH-7607a50e-2023-5,2023,Energy,Naphta,GJ,6344.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,SRC_AlamosGoldInc_Data_2023,6.344000e+06,direct_unit,gj→MJ factor=1000.0,MJ,False
229,TECH-7607a50e-2023-6,2023,Energy,Natural gas,GJ,221612.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,SRC_AlamosGoldInc_Data_2023,2.216120e+08,direct_unit,gj→MJ factor=1000.0,MJ,False


In [19]:
energy_df_sd.to_excel(r'data/MetalliCan/pre_cleaned_data/energy_df.xlsx', index=False)

## Material flows

In [20]:
from core.conversion_functions import standardize_materials_to_t

In [21]:
material_df_sd = standardize_materials_to_t(material_df)

In [22]:
material_df_sd

,technosphere_id,year,flow_type,subflow_type,unit,value,comment,main_id,facility_group_id,company_id,source_id,mass_t,mass_source,mass_note,needs_density
35,TECH-6dc537e6-2023-2,2023,Material use,Cement,t,27374.000000,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,27374.000000,t,reported in tonnes,False
39,TECH-6dc537e6-2023-6,2023,Material use,Lime,t,2229.000000,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,2229.000000,t,reported in tonnes,False
40,TECH-6dc537e6-2023-7,2023,Material use,Lubricants,kl,380.687000,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,335.004560,L×density→t,kl→L × density kg/L; density=0.88,False
41,TECH-6dc537e6-2023-8,2023,Material use,Nitric acid (HNO3),l,26575.000000,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,NaN,missing_density,volume reported; no density mapping for this s...,True
43,TECH-6dc537e6-2023-10,2023,Material use,Sodium cyanide (NaCN),t,838.800000,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,838.800000,t,reported in tonnes,False
44,TECH-6dc537e6-2023-11,2023,Material use,Sulfuric acid (H2SO4),l,73.960000,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,NaN,missing_density,volume reported; no density mapping for this s...,True
45,TECH-6dc537e6-2023-12,2023,Material use,Tires,t,241.250567,NaN,QC-MAIN-6dc537e6,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,241.250567,t,reported in tonnes,False
65,TECH-687b8c8d-2023-6,2023,Material use,Total sodium cyanide used,t,245.000000,NaN,ON-MAIN-687b8c8d,NaN,CMP-3a4ccc7f,SRC_AlamosGoldInc_Data_2023,245.000000,t,reported in tonnes,False
66,TECH-687b8c8d-2023-7,2023,Material use,Total blasting agents used e.g. ANFO,t,486.300000,NaN,ON-MAIN-687b8c8d,NaN,CMP-3a4ccc7f,SRC_AlamosGoldInc_Data_2023,486.300000,t,reported in tonnes,False
116,TECH-fefeaee4-2023-3,2023,Material use,Cement,t,9417.600000,NaN,ON-MAIN-fefeaee4,NaN,CMP-4a434d72,SRC_NewmontCorporation_Newmont-2023-Performanc...,9417.600000,t,reported in tonnes,False


In [23]:
material_df_sd.to_excel(r'data/MetalliCan/pre_cleaned_data/material_df.xlsx', index=False)

# Normalization

In [26]:
from core.data_manipulations import normalize_by_production

In [27]:
normalization_production_df = pd.read_excel(r'data/MetalliCan/sites_for_lci.xlsx', sheet_name='prod_data')

In [30]:
energy_df_norm = normalize_by_production(energy_df_sd, normalization_production_df, value_col='value_MJ', prod_col='ore_processed_t')

In [33]:
material_df_norm = normalize_by_production(material_df_sd, normalization_production_df, value_col='mass_t', prod_col='ore_processed_t')

In [34]:
biosphere_df_norm = normalize_by_production(env_table, normalization_production_df, value_col='value', prod_col='ore_processed_t')

In [37]:
energy_df_norm.to_csv(r'data/MetalliCan/cleaned_data/energy_df.csv', index=False)
material_df_norm.to_csv(r'data/MetalliCan/cleaned_data/material_df.csv', index=False)
biosphere_df_norm.to_csv(r'data/MetalliCan/cleaned_data/biosphere_df.csv', index=False)

In [32]:
biosphere_df_norm['unit_normalized'] = biosphere_df_norm['unit'] + '/t'

# Clean land occupation table

In [33]:
land_table

,land_occupation_id,area_km2,geometry,distance_km,main_id,tailing_id,source_id
0,45954,2.622800,POLYGON Z ((-62.920418000502096 44.98665598524...,0.000000,NS-MAIN-c9c77a59,NaN,https://doi.org/10.1038/s43247-023-00805-6
1,45955,0.441869,POLYGON Z ((-62.92612599959345 44.991678995543...,1.186584,NS-MAIN-c9c77a59,NaN,https://doi.org/10.1038/s43247-023-00805-6
2,7631,0.209038,POLYGON Z ((-53.8260233402639 47.4043112260747...,0.811781,NL-MAIN-d9036091,NaN,https://doi.org/10.1038/s43247-023-00805-6
3,7641,0.113569,POLYGON Z ((-53.7468445297622 47.4086824659858...,4.647047,NL-MAIN-d9036091,NaN,https://doi.org/10.1038/s43247-023-00805-6
4,7633,0.213592,POLYGON Z ((-53.81396413042625 47.413655936064...,0.000000,NL-MAIN-d9036091,NaN,https://doi.org/10.1038/s43247-023-00805-6
...,...,...,...,...,...,...,...
789,QC-LI-219a2e44,2.926809,MULTIPOLYGON Z (((-79.08481557072447 48.229685...,NaN,QC-MAIN-30c1828c,NaN,https://www.nature.com/articles/s41597-025-052...
790,BC-LI-9c3aa818,0.122644,POLYGON Z ((-130.49619675383414 56.64850845409...,NaN,BC-MAIN-b1fe389a,NaN,https://www.nature.com/articles/s41597-025-052...
791,ON-LI-1a9a061c,4.202254,MULTIPOLYGON Z (((-81.36899483562593 48.678821...,NaN,ON-MAIN-f8313ebd,NaN,https://www.nature.com/articles/s41597-025-052...
792,QC-LI-94bd8222,6.202897,POLYGON Z ((-79.22195784417586 49.574666146450...,NaN,QC-MAIN-b86f7d07,NaN,https://www.nature.com/articles/s41597-025-052...


In [34]:
# Step 1: Find main_id that have more than one unique source_id
multi_source_main_ids = land_table.groupby('main_id')['source_id'].nunique()
multi_source_main_ids = multi_source_main_ids[multi_source_main_ids > 1].index

# Step 2: Filter the dataframe
land_table = land_table[
    ~land_table['main_id'].isin(multi_source_main_ids) |
    (land_table['source_id'] == "https://www.nature.com/articles/s41597-025-05296-y")
]

In [35]:
land_table

,land_occupation_id,area_km2,geometry,distance_km,main_id,tailing_id,source_id
0,45954,2.622800,POLYGON Z ((-62.920418000502096 44.98665598524...,0.000000,NS-MAIN-c9c77a59,NaN,https://doi.org/10.1038/s43247-023-00805-6
1,45955,0.441869,POLYGON Z ((-62.92612599959345 44.991678995543...,1.186584,NS-MAIN-c9c77a59,NaN,https://doi.org/10.1038/s43247-023-00805-6
2,7631,0.209038,POLYGON Z ((-53.8260233402639 47.4043112260747...,0.811781,NL-MAIN-d9036091,NaN,https://doi.org/10.1038/s43247-023-00805-6
3,7641,0.113569,POLYGON Z ((-53.7468445297622 47.4086824659858...,4.647047,NL-MAIN-d9036091,NaN,https://doi.org/10.1038/s43247-023-00805-6
4,7633,0.213592,POLYGON Z ((-53.81396413042625 47.413655936064...,0.000000,NL-MAIN-d9036091,NaN,https://doi.org/10.1038/s43247-023-00805-6
...,...,...,...,...,...,...,...
789,QC-LI-219a2e44,2.926809,MULTIPOLYGON Z (((-79.08481557072447 48.229685...,NaN,QC-MAIN-30c1828c,NaN,https://www.nature.com/articles/s41597-025-052...
790,BC-LI-9c3aa818,0.122644,POLYGON Z ((-130.49619675383414 56.64850845409...,NaN,BC-MAIN-b1fe389a,NaN,https://www.nature.com/articles/s41597-025-052...
791,ON-LI-1a9a061c,4.202254,MULTIPOLYGON Z (((-81.36899483562593 48.678821...,NaN,ON-MAIN-f8313ebd,NaN,https://www.nature.com/articles/s41597-025-052...
792,QC-LI-94bd8222,6.202897,POLYGON Z ((-79.22195784417586 49.574666146450...,NaN,QC-MAIN-b86f7d07,NaN,https://www.nature.com/articles/s41597-025-052...


In [36]:
# For each main_id, give me the sum of area_km2 associated
land_table = land_table.groupby('main_id')['area_km2'].sum().reset_index()

In [37]:
land_table

,main_id,area_km2
0,BC-MAIN-23155c25,1.499690
1,BC-MAIN-3ef4f421,1.396089
2,BC-MAIN-3f490561,7.967835
3,BC-MAIN-4724f4ba,0.416737
4,BC-MAIN-599152a0,13.233210
...,...,...
111,SK-MAIN-9dd2b7f8,4.345047
112,SK-MAIN-bb89158f,10.235647
113,SK-MAIN-d3c471e8,1.973892
114,YT-MAIN-44857446,5.293594


In [38]:
land_table = land_table.merge(main_table[['main_id', 'facility_group_id', 'facility_name', 'facility_group_name', 'province', 'facility_type', 'mining_processing_type', 'commodities']], on='main_id', how='left')

In [39]:
land_table

,main_id,area_km2,facility_group_id,facility_name,facility_group_name,province,facility_type,mining_processing_type,commodities
0,BC-MAIN-23155c25,1.499690,NaN,Myra Falls,NaN,British Columbia,mining,Underground,"Zinc, copper, silver, gold, lead"
1,BC-MAIN-3ef4f421,1.396089,NaN,Avanti Kitsault,NaN,British Columbia,project,NaN,"Molybdenum, silver, lead"
2,BC-MAIN-3f490561,7.967835,NaN,Mount Polley,NaN,British Columbia,mining,"Open-pit, concentrator","Gold, copper, silver"
3,BC-MAIN-4724f4ba,0.416737,NaN,Elk,NaN,British Columbia,mining,Open-pit,Gold
4,BC-MAIN-599152a0,13.233210,NaN,Copper Mountain,NaN,British Columbia,mining,"Open-pit, concentrator","Copper, gold, silver"
...,...,...,...,...,...,...,...,...,...
111,SK-MAIN-9dd2b7f8,4.345047,NaN,Horseshoe-Raven,NaN,Saskatchewan,project,NaN,Uranium
112,SK-MAIN-bb89158f,10.235647,GRP-21eee27d,Key Lake,Key Lake + McArthur River,Saskatchewan,mining,Concentrator,Uranium
113,SK-MAIN-d3c471e8,1.973892,GRP-21eee27d,McArthur River,Key Lake + McArthur River,Saskatchewan,mining,Underground,Uranium
114,YT-MAIN-44857446,5.293594,NaN,Keno Hill Silver District,NaN,Yukon,mining,"Underground, concentrator","Silver, zinc, lead"


In [40]:
land_table_mining = land_table[land_table['facility_type'] == 'mining']

In [41]:
land_table_mining.to_excel(r'data/MetalliCan/cleaned_data/land_table_mining.xlsx', index=False)

# Add commodity information

In [42]:
energy_df_sd_norm = merge_main_and_group(energy_df_sd_norm, main_table)
material_df_sd_norm = merge_main_and_group(material_df_sd_norm, main_table)
biosphere_df_norm = merge_main_and_group(biosphere_df_norm, main_table)

In [43]:
energy_df_sd_norm

,technosphere_id,year,flow_type,subflow_type,unit,value,comment,main_id,facility_group_id,company_id,...,needs_factor,value_tonnes_main,value_tonnes_group,value_tonnes_match,value_normalized,normalization_key,facility_name,facility_group_name,mining_processing_type,commodities
0,TECH-857b7b89-2023-1,2023,Energy,Acetylene,GJ,18.475651,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,...,False,166000.0,NaN,166000.0,0.111299,main_id,Brucejack,NaN,"Underground, concentrator","Gold, silver"
1,TECH-857b7b89-2023-2,2023,Energy,Aviation fuel,GJ,72676.110790,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,...,False,166000.0,NaN,166000.0,437.807896,main_id,Brucejack,NaN,"Underground, concentrator","Gold, silver"
2,TECH-857b7b89-2023-3,2023,Energy,Diesel,GJ,287042.447232,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,...,False,166000.0,NaN,166000.0,1729.171369,main_id,Brucejack,NaN,"Underground, concentrator","Gold, silver"
3,TECH-857b7b89-2023-4,2023,Energy,Gasoline,GJ,13568.450000,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,...,False,166000.0,NaN,166000.0,81.737651,main_id,Brucejack,NaN,"Underground, concentrator","Gold, silver"
4,TECH-857b7b89-2023-5,2023,Energy,Propane,GJ,42071.041300,NaN,BC-MAIN-857b7b89,NaN,CMP-4a434d72,...,False,166000.0,NaN,166000.0,253.440008,main_id,Brucejack,NaN,"Underground, concentrator","Gold, silver"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,TECH-7607a50e-2023-3,2023,Energy,Diesel,GJ,140100.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,...,False,2878047.0,NaN,2878047.0,48.678844,main_id,Young-Davidson,NaN,"Underground, concentrator",Gold
178,TECH-7607a50e-2023-4,2023,Energy,Gasoline,GJ,2124.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,...,False,2878047.0,NaN,2878047.0,0.738000,main_id,Young-Davidson,NaN,"Underground, concentrator",Gold
179,TECH-7607a50e-2023-5,2023,Energy,Naphta,GJ,6344.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,...,False,2878047.0,NaN,2878047.0,2.204273,main_id,Young-Davidson,NaN,"Underground, concentrator",Gold
180,TECH-7607a50e-2023-6,2023,Energy,Natural gas,GJ,221612.000000,NaN,ON-MAIN-7607a50e,NaN,CMP-3a4ccc7f,...,False,2878047.0,NaN,2878047.0,77.000827,main_id,Young-Davidson,NaN,"Underground, concentrator",Gold


In [45]:
energy_df_sd_norm.to_excel(r'data/MetalliCan/cleaned_data/energy_df_sd_norm.xlsx', index=False)
material_df_sd_norm.to_excel(r'data/MetalliCan/cleaned_data/material_df_sd_norm.xlsx', index=False)
biosphere_df_norm.to_excel(r'data/MetalliCan/cleaned_data/biosphere_df_norm.xlsx', index=False)

# Selecting samples of facilities

In [42]:
ids_energy = set(energy_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_material = set(material_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_biosphere = set(biosphere_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_land = set(land_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))

In [43]:
# Facilities with at least technosphere and biosphere flows
#best_ids = ids_energy & ids_material & ids_biosphere
#best_ids

{('BC-MAIN-aa76f6f2', nan),
 ('ON-MAIN-0aadf28f', nan),
 ('ON-MAIN-687b8c8d', nan),
 ('ON-MAIN-7607a50e', nan),
 ('ON-MAIN-fefeaee4', nan),
 ('QC-MAIN-02884fb5', nan),
 ('QC-MAIN-6dc537e6', nan),
 (nan, 'GRP-0d911886')}

In [60]:
second_best_ids = ids_energy & ids_biosphere
second_best_ids

{('BC-MAIN-599152a0', nan),
 ('BC-MAIN-6b4800fe', nan),
 ('BC-MAIN-857b7b89', nan),
 ('BC-MAIN-8eb8be0d', nan),
 ('BC-MAIN-aa76f6f2', nan),
 ('BC-MAIN-ed23117f', nan),
 ('NL-MAIN-b64bae7a', nan),
 ('NL-MAIN-dd723db4', nan),
 ('NU-MAIN-8b0264c9', nan),
 ('ON-MAIN-0aadf28f', nan),
 ('ON-MAIN-1f126a43', nan),
 ('ON-MAIN-687b8c8d', nan),
 ('ON-MAIN-6e9be24e', nan),
 ('ON-MAIN-7607a50e', nan),
 ('ON-MAIN-7f050560', nan),
 ('ON-MAIN-a3c56a83', nan),
 ('ON-MAIN-aeafbb59', nan),
 ('ON-MAIN-cb85213a', nan),
 ('ON-MAIN-fefeaee4', nan),
 ('QC-MAIN-02884fb5', nan),
 ('QC-MAIN-6dc537e6', nan),
 ('QC-MAIN-9de9bb0d', nan),
 ('QC-MAIN-b86f7d07', nan),
 ('QC-MAIN-c0660aec', nan),
 ('QC-MAIN-e51eda66', nan),
 ('QC-MAIN-e7e6a960', nan),
 ('QC-MAIN-f9e41c2a', nan),
 ('YT-MAIN-44857446', nan),
 (nan, 'GRP-0a2c0d69'),
 (nan, 'GRP-0d911886'),
 (nan, 'GRP-147b3123'),
 (nan, 'GRP-14bfbb82'),
 (nan, 'GRP-a13779f8')}

In [61]:
main_df_second = get_info_for_ids(main_table, second_best_ids)
main_df_second = main_df_second[['main_id', 'facility_group_id', 'facility_name', 'facility_group_name', 'province', 'facility_type', 'mining_processing_type', 'commodities']]
main_df_second

,main_id,facility_group_id,facility_name,facility_group_name,province,facility_type,mining_processing_type,commodities
3,NU-MAIN-4ea8ac9d,GRP-0a2c0d69,Amaruq,Meadowbank complex,Nunavut,mining,"Open-pit, underground",Gold
4,QC-MAIN-e7e6a960,NaN,Canadian Malartic,NaN,Quebec,mining,"Open-pit, concentrator","Gold, silver"
5,ON-MAIN-aeafbb59,NaN,Detour Lake,NaN,Ontario,mining,"Open-pit, concentrator",Gold
6,QC-MAIN-c0660aec,NaN,Goldex,NaN,Quebec,mining,"Underground, concentrator","Gold, silver"
7,QC-MAIN-e51eda66,NaN,LaRonde,NaN,Quebec,mining,"Underground, concentrator","Gold, zinc, copper, silver, cadmium"
8,ON-MAIN-1f126a43,NaN,Macassa,NaN,Ontario,mining,"Underground, concentrator","Gold, silver"
9,NU-MAIN-730aefe3,GRP-0a2c0d69,Meadowbank,Meadowbank complex,Nunavut,mining,Concentrator,Gold
10,NU-MAIN-8b0264c9,NaN,Meliadine,NaN,Nunavut,mining,"Open-pit, underground, concentrator",Gold
13,ON-MAIN-687b8c8d,NaN,Island,NaN,Ontario,mining,"Underground, concentrator",Gold
14,ON-MAIN-7607a50e,NaN,Young-Davidson,NaN,Ontario,mining,"Underground, concentrator",Gold
